In [12]:
# ML libraries
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# web scraping
import bs4 as bs
import urllib.request


In [13]:
def create_df_and_similarty():
    df = pd.read_csv('all_movies.csv')
    # creating a count matrix
    cv = CountVectorizer()
    count_matrix = cv.fit_transform(df['all_cast'].values.astype('U'))
    # creating a similarity score matrix
    similarity = cosine_similarity(count_matrix)
    return df, similarity

In [14]:

def recommend_movie(movie):
    movie = movie.lower()

    df, similarity = create_df_and_similarty()
    if movie not in df['movie_title'].unique():
        return('Sorry, there is no such a movie')
    else:
        ind = df.loc[df['movie_title']==movie].index[0]
        lst = list(enumerate(similarity[ind]))
        lst = sorted(lst, key=lambda x:x[1], reverse=True)
        top_10 = lst[1:11]
        recommended_movies = []
        for i in range(len(top_10)):
            
            recommended_movies.append(df['movie_title'][top_10[i][0]])
        
        return recommended_movies

In [55]:
recommend_movie('underwater')

['the strange ones',
 'stratton',
 'sweet country',
 'the commuter',
 'proud mary',
 'acts of violence',
 'freak show',
 'humor me',
 'vazante',
 "mary and the witch's flower"]

In [56]:
recommend_movie('hisnbfn')

'Sorry, there is no such a movie'

In [45]:
import pickle
import requests
from tmdbv3api import TMDb
tmdb = TMDb()
tmdb.api_key = '2239848dd3e0d8e43cf2add39e687057'
from tmdbv3api import Movie
 

In [46]:
file = 'nlp.pkl'
RFC = pickle.load(open(file, 'rb'))
vectorizer = pickle.load(open('tranform.pkl','rb'))

In [52]:
 def get_reviews(movie):
        tmdb_movie = Movie()
        output = tmdb_movie.search(movie)
        # get the movie id and  title
        movie_id = output[0].id
        movie_name = output[0].title

        # get the response using the API
        response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id, tmdb.api_key))
        response_json = response.json()
        imdb_id = response_json['imdb_id']
        
    
        # web scraping to get user reviews from the IMDB 
        sauce = urllib.request.urlopen('https://www.imdb.com/title/{}/reviews?ref_=tt_ov_rt'.format(imdb_id)).read()
        soup = bs.BeautifulSoup(sauce, 'lxml')
        soup_result = soup.find_all("div", {"class": "text show-more__control"})

        reviews_list = []  # list of reviews
        reviews_status = []  # list of comments
        for reviews in soup_result:
            if reviews.string:
                reviews_list.append(reviews.string)
                # passing the review to our model
                movie_review_list = np.array([reviews.string])
                movie_vector = vectorizer.transform(movie_review_list)
                pred = RFC.predict(movie_vector)
                reviews_status.append('Amazing' if pred else 'Not so good')
                # combining reviews and comments into dictionary
            movie_reviews = {reviews_list[i]: reviews_status[i] for i in range(len(reviews_list))}
        return  movie_reviews

In [53]:
get_reviews('I, Robot')

{"The film shares the same name as Asimov's novel, but with some adaptations. But in general, it doesn't break the three laws. This kind of science fiction film, which integrates human nature and ethics of science and technology, triggers our deep reflection. Human beings and robots have advantages and disadvantages, no matter when, in front of any biological species, we always have the freedom to choose.": 'Amazing',
 '"I, Robot" was this year\'s most entertaining Blockbuster: a lot of action, witty dialog and a fine plot. In fact, I thought the plot was so good that it was a waste to turn this project into a Will Smith vehicle. Not that he ruined the movie or anything but his funnyman performance - and the witty dialog - just kept the story from getting deep. So many other recent Blockbusters were unnecessarily lofty (*cough*SPIDER-MAN2*cough*MATRIX2&3*cough*cough*) and could have used some irony. With "I, Robot", however, it would have been a great idea to go for a more philosophica

In [63]:
def get_movie_with_reviews(movie):
    recommended_movies = recommend_movie(movie)
    pairs = []
    for m in recommended_movies:
        rev = get_reviews(m)
        pairs.append([m, rev])
    return pairs


In [64]:
pairs = get_movie_with_reviews("X-Men: Days of Future Past")

In [67]:
movie_1, review_1 = pairs[1]
movie_1

'x-men: apocalypse'

In [68]:
review_1

{'Suffering from bizarre narrative choices and weak character development, X-Men: Apocalypse is an otherwise passable and occasionally great installment of the franchise, mainly due to the continued strong performances and glorious action set pieces.': 'Amazing',
 "X-Men: Apocalypse is the sequel to Days of Future Past and takes place  about a decade later in the '80s. This movie entertained me from start  to finish and I can't say I was ever bored. But there are a lot of  things I was not satisfied with while watching. Starting off with the  good, Magneto and Professor X continue to be the best parts of these  movies. Michael Fassbender as Magneto is amazing and he has a really  emotional scene towards the beginning which got me locked into the  film. James McAvoy is also great as Xavier. The internal battles inside  Xavier's head were riveting. I really liked the new cast of the younger  X-Men, like Jean Grey, Storm, Nightcrawler, and Cyclops. Quicksilver  has another awesome scene j

In [69]:
movie_0, review_0 = pairs[0]

In [70]:
movie_0

'the chronicles of narnia: prince caspian'

In [71]:
review_0

{"I am a Narnia obsessed nerd, so of course I went to see it the night it was released. I had very high expectations because the first movie was so true to the book. However, with this film that wasn't the case. The screen play was pretty much completely rewritten and included only hints of the original story. I suppose Disney felt it necessary to add in lame typical movie elements that attract mainstream moviegoers. For example a sub-plot romance between Caspian and Susan and battles that never existed. This movie not only failed to develop the characters and events as well as the book, it changed the characters and situations that it did develop.": 'Amazing',
 'Since the Chronicles of Narnia are a series of widely read and revered books I had to give my first rating of a 1 for this film. We went opening night and I was very disappointed and my wife was in tears by the end because of her disappointment. So much of Lewis\' humor was lost in favor of an action movie plot with running ba

In [72]:
movie_2,review_2 = pairs[2]

In [73]:
movie_2

'x-men: first class'

In [74]:
review_2

{"With Matthew Vaughn at the helm, X-Men: First Class is a classy, fun time that brings the best out of it's strong cast. The action scenes and dialogue pop with a ferocity, and breaths life into a incredibly engaging storyline.": 'Amazing',
 "What you need to know about this movie before you go and see it.  1. Michael Fassbender and James McAvoy are superb. Each one, alone,  would have made the movie terrific, but together? Forgetaboutit! Every  step of the way, every scene, every emotion, these two men own the  movie and watching their bromance is a treat from start to finish. Make  no mistake about it, even when the world is on the brink of utter  destruction what really matters is how these two men relate to one  another. And when you watch the movie, you will care.  2. Everyone in the cast is good. Kevin Bacon has never been better, and  Jennifer Lawrence is wonderful. Rose Byrne and January Jones aren't  given much to do, Ray Wise and Michael Ironside only have a few short  lines

In [76]:
recommend_movie("X-Men: Days of Future Past")

['the chronicles of narnia: prince caspian',
 'x-men: apocalypse',
 'x-men: first class',
 'the hunger games: catching fire',
 'ice age: continental drift',
 'pixels',
 'the hunger games',
 'the angry birds movie',
 'american hustle',
 'elf']